In [ ]:
# Load pacakges ----
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import HTML
from IPython.display import Markdown as md
from IPython.core.display import HTML as Center
from config import marker, qof_measure_marker, start_date, end_date, demographics, codelist_path
from IPython.display import Image, display

from utilities import *
%matplotlib inline

In [ ]:
display(
    md(f"## Changes in {marker} between {start_date} and {end_date}"),
    md(f"### Indicator: {qof_measure_marker} (Version: 46.0)")
)

The denominator and numerator rules for BP002 (Version: 46.0) are outlined beloow.
Prevalence data is not available for the blood pressure indicator. 
More details can be found [here](https://digital.nhs.uk/data-and-information/data-collections-and-data-sets/data-collections/quality-and-outcomes-framework-qof/quality-and-outcome-framework-qof-business-rules/qof-business-rules-v46.0-2021-2022-baseline-release).
Dashboards presenting the annual targets of all QOF indicators published by NHSD are available [here](https://digital.nhs.uk/data-and-information/data-tools-and-services/data-services/general-practice-data-hub/quality-outcomes-framework-qof).

#### Denominator

| Rule number | Rule | Action if true | Action if false | Rule description or comments |
|:---|:---|:---|:---|:---| 
| 1 | If PAT_AGE < 45 | Reject | Next rule | Reject patients from the specified population who are aged less than 45 years old. Pass all remaining patients to the next rule. |
| 2 | If BP_DAT > (PPED – 5 years) | Select | Next rule | Select patients passed to this rule who had their blood pressure recorded in the 5 year period leading up to and including the payment period end date. Pass all remaining patients to the next rule. |
| 3 | If BPDEC_DAT > (PPED – 5 years)  | Reject | Next rule | Reject patients passed to this rule chose not to have their blood pressure recorded in the 5 year period leading up to and including the payment period end date. Pass all remaining patients to the next rule. |
| 4 | If REG_DAT > (PPED – 3 months) | Reject | Select | Reject patients passed to this rule who registered with the GP practice in the 3 month period leading up to and including the payment period end date. Select the remaining patients. |

#### Numerator

| Rule number | Rule | Action if true | Action if false | Rule description or comments |
|:---|:---|:---|:---|:---| 
| 1 | If BP_DAT > (PPED – 5 years) | Select | Reject | Select patients from the denominator who had their blood pressure recorded in the 5 year period leading up to and including the payment period end date. Reject the remaining patients. |



In [ ]:
display(
    md("### Methods"),
    md(f"Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed coding activity related to {marker} (Indicator: {qof_measure_marker}) between {start_date} and {end_date}. The codelist used can be found here at [OpenSAFELY Codelists](https://codelists.opensafely.org/)."),
    md("For each month within the study period, we have calculated the percentage of registered patients above 45 years that had their blood pressure recorded in the 5 year period leading up to the month of reporting."),
    md(f"All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](https://github.com/opensafely/blood-pressure-qof)")
)

In [ ]:
codelist = pd.read_csv(f'../{codelist_path}')

image_paths = {d: f'../output/plot_{d}.png' for d in demographics}
image_paths['total'] = '../output/plot_total.png'

In [ ]:
md(f"## Total achievement of {marker} over time")

In [ ]:
Image(filename=image_paths['total'])


In [ ]:
display(
    md("### Sub totals by sub codes"),
    md("Events for the top 5 subcodes across the study period")
)

In [ ]:
child_table = pd.read_csv('../output/child_code_table.csv')
child_table

In [ ]:
md("## Total Number by GP Practice")

In [ ]:
practice_table = pd.read_csv('../output/rate_table_practice.csv', parse_dates=['date']).sort_values(by='date')
percentage_practices = get_percentage_practices(practice_table)
md(f"Percentage of practices with a recording of a code within the codelist during the study period: {percentage_practices}%")

In [ ]:
Image(filename='../output/decile_chart.png')

In [ ]:
i = 0
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'") 

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")  

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
display(Image(filename=image_paths[demographics[i]]))

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]]) 